# 1 pacotes 

In [ ]:
import mwclient
import time
import datetime as dt
from datetime import date, timedelta
import pandas as pd
import json

## 2 Obter MatchId

In [60]:
def obterMatchId(dataJogo:str,**kwargs):
    
    liga = kwargs.get('liga','')
    dataJogoFim = kwargs.get('dataJogoFim','')
    
    date = dt.datetime.strptime(dataJogo, "%Y-%m-%d").date()
    dateFinal = dt.datetime.strptime(dataJogoFim, "%Y-%m-%d").date() if len(dataJogoFim)>0 else date
    
    cond = "M.DateTime_UTC >= '" + str(date) + " 00:00:00' AND M.DateTime_UTC <= '" + str(dateFinal+dt.timedelta(1)) + " 00:00:00'"
    if len(liga)>0:
        cond = cond+" AND "+f"M.MatchId like '%{liga}%'"
    
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = 'max',
        tables = "MatchSchedule=M",
        fields = "M.MatchId,M.DateTime_UTC,M.PageAndTeam1, M.PageAndTeam2, M.Team1,M.Team2, M.BestOf",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [62]:
obterMatchId('2022-07-02', liga = 'CBLOL',dataJogoFim='2022-07-03')

,MatchId,DateTime UTC,PageAndTeam1,PageAndTeam2,Team1,Team2,BestOf,DateTime UTC__precision
0,CBLOL/2022 Season/Split 2_Week 4_1,2022-07-02 16:00:00,CBLOL/2022 Season/Split 2_Miners,CBLOL/2022 Season/Split 2_Flamengo Los Grandes,Miners,Flamengo Los Grandes,1,0
1,CBLOL/2022 Season/Split 2_Week 4_10,2022-07-03 20:00:00,CBLOL/2022 Season/Split 2_Flamengo Los Grandes,CBLOL/2022 Season/Split 2_FURIA Esports,Flamengo Los Grandes,FURIA Esports,1,0
2,CBLOL/2022 Season/Split 2_Week 4_2,2022-07-02 17:00:00,CBLOL/2022 Season/Split 2_Rensga Esports,CBLOL/2022 Season/Split 2_LOUD,Rensga Esports,LOUD,1,0
3,CBLOL/2022 Season/Split 2_Week 4_3,2022-07-02 18:00:00,CBLOL/2022 Season/Split 2_RED Canids,CBLOL/2022 Season/Split 2_KaBuM! e-Sports,RED Canids,KaBuM! e-Sports,1,0
4,CBLOL/2022 Season/Split 2_Week 4_4,2022-07-02 19:00:00,CBLOL/2022 Season/Split 2_paiN Gaming,CBLOL/2022 Season/Split 2_FURIA Esports,paiN Gaming,FURIA Esports,1,0
5,CBLOL/2022 Season/Split 2_Week 4_5,2022-07-02 20:00:00,CBLOL/2022 Season/Split 2_INTZ,CBLOL/2022 Season/Split 2_Liberty,INTZ,Liberty,1,0
6,CBLOL/2022 Season/Split 2_Week 4_6,2022-07-03 16:00:00,CBLOL/2022 Season/Split 2_KaBuM! e-Sports,CBLOL/2022 Season/Split 2_Rensga Esports,KaBuM! e-Sports,Rensga Esports,1,0
7,CBLOL/2022 Season/Split 2_Week 4_7,2022-07-03 17:00:00,CBLOL/2022 Season/Split 2_Liberty,CBLOL/2022 Season/Split 2_RED Canids,Liberty,RED Canids,1,0
8,CBLOL/2022 Season/Split 2_Week 4_8,2022-07-03 18:00:00,CBLOL/2022 Season/Split 2_INTZ,CBLOL/2022 Season/Split 2_paiN Gaming,INTZ,paiN Gaming,1,0
9,CBLOL/2022 Season/Split 2_Week 4_9,2022-07-03 19:00:00,CBLOL/2022 Season/Split 2_LOUD,CBLOL/2022 Season/Split 2_Miners,LOUD,Miners,1,0


## Obter GameId

In [46]:
def obterGameId(macthid):
    
    cond = f"MG.MatchId = '{macthid}'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = '10',
        tables = "MatchScheduleGame=MG",
        fields = "MG.GameId",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [51]:
def obterGameIdv2(macthid):
    
    cond = f"SG.MatchId like '{macthid}'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = '10',
        tables = "ScoreboardGames=SG",
        fields = "SG.GameId",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [50]:
obterGameId('CBLOL/2022 Season/Split 2_Week 4_10')

,GameId
0,CBLOL/2022 Season/Split 2_Week 4_10_1


In [52]:
obterGameIdv2('CBLOL/2022 Season/Split 2_Week 4_10')

,GameId
0,CBLOL/2022 Season/Split 2_Week 4_10_1


In [ ]:
b                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          